### Assignment 10

In this assignment you will experiment on your own. Using a health dataset of your choice (check with us if you are not sure), write code to demonstrate the following Pandas functions:

- Melt
- Pivot
- Aggregation
- Iteration
- Groupby

Here are some datasets you can use if you don’t have one:
- https://archive.ics.uci.edu/ml/datasets/Breast+Cancer
- https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008 --I'm choosing this one
- https://archive.ics.uci.edu/ml/datasets/Arrhythmia

Each function demonstration will be for 30 points for a total of 150 points. Ensure that you include comments within your code and follow the rubric as a guide. Submit using your GitHub site. Ask if you have any questions.

In [1]:
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('diabetic_data.csv')
diabetes = pd.DataFrame(diabetes)
diabetes.info()
diabetes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
#We are going to explore (1) data types in the object columns and (2) What this string data entails (categorical, binary, nominal, etc).

#The most common reason for an 'object' dtype is that the column holds variable types of data. 
#Pandas is known for using 'object' as a catch-all for columns that don't fit into a specific numeric or string type. However, a quirk of Pandas is ALSO to label "string" datatypes as "objects."

diabetes_expl = diabetes.copy()

obj_col_metadata = []
obj_col_md_valstr = [] #These will allow us to inspect the stats of the object columns - what datatypes are in there? How many? of what string values? Because we're going to make them consistent and possibly convert them to binary if it makes sense to do so.

obj_cols = diabetes_expl.select_dtypes(include='object').columns #we are batch-isolating the 'object' columns to see what's in there
for n in obj_cols: #for each object column
    t2d_object_types = diabetes_expl[n].apply(type).value_counts() #type - find the datatypes in/of the object columns and how many of each type
    for dtype, count in t2d_object_types.items(): #and ... 
        obj_col_metadata.append({'column': n, 'data_type': str(dtype), 'dt_count': count}) #put the types and type-counts for each column in the empty list
    
    t2d_object_values = diabetes_expl[n].value_counts(dropna=False) #value_counts - list the specific values (assuming they are at least categorical) and how many of each there are. We care about "how many" mostly for potential conversion or manipulation, not yet for data analysis.
    for val, count in t2d_object_values.items(): #and ... 
        obj_col_metadata.append({'column': n, 'val_type': str(val), 'val_count': count}) #put the values and value-counts for each column in the same list. 'columns' will be overwritten by this second pass, but that's totally fine. 

    t2d_object_values_2 = diabetes_expl[n].value_counts(dropna=False) #Another organization of the same information. A bit shorter to read.
    value_summary = "; ".join([f"{str(val)} ({count})" for val, count in t2d_object_values_2.items()])
    obj_col_md_valstr.append({'column': n, 'data_type': str(dtype), 'dt_count': count, 'value summary': value_summary})


# Convert the output to a DataFrame, so we can then ... 
obj_col_metadata_df = pd.DataFrame(obj_col_metadata)
obj_col_md_valstr_df = pd.DataFrame(obj_col_md_valstr)

# ... export to CSV
obj_col_metadata_df.to_csv('obj_col_metadata.csv', index=False)
obj_col_md_valstr_df.to_csv('obj_col_md_valstr.csv', index=False)

We're going to deal with Object datatypes and turn them ALL to Strings. 

In [4]:
#max_glu_serum and A1Cresult are Object classes comprising 2 datatypes:  strings AND floats. But the floats are all nan? Well that's too bad, the floats are just not available. Weird that Python registers that as float columns though. 
#All other Object classes are only strings

obj_cols2 = diabetes.select_dtypes(include='object').columns
for n in obj_cols2:
    if n in diabetes.columns:
        diabetes[n] = diabetes[n].astype("string")

print(diabetes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  string
 3   gender                    101766 non-null  string
 4   age                       101766 non-null  string
 5   weight                    101766 non-null  string
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  string
 11  medical_specialty         101766 non-null  string
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

Now we're going to deal with missing values (or null, blank, NaN, not applicable, not recorded, unknown). They are NOT consistently rendered. I am using the .csv files I generated to explore the values that are possible.

In [5]:
# Uses nan: max_glu_serum and A1Cresult, as we noted above when dealing with object types. But the nan values were formerly floats, which is too bad, as those measures are just not available.
# Uses ?: Race, weight, payer_code, medical_specialty, diag_1, diag_2, and diag_3
# Uses Unknown/Invalid: gender

# I understand what they're trying to convey for gender, and ... 
# even though I HATE the ?, I don't want to expand the field by replacing it with "Unknown", especially if the code fields are limited in size. 
# Alternatively replacing ? with a blank or NaN would be misleading in some cases. 

# I think ... I think the only thing I'm going to change is the text 'nan' to an actual .isna value in max_glu_serum and A1Cresult. Not really necessary, but I don't want a 'fake' NaN.

NaN_cols = ["max_glu_serum", 'A1Cresult']

for n in NaN_cols:
    if n in diabetes.columns:
        diabetes[n] = diabetes[n].replace('nan',np.nan)


We're going to check if encounter_id or patient_nbr are unique.

In [6]:

unique_encounters = diabetes['encounter_id'].nunique()
print(f"There are {unique_encounters} unique encounters in the dataset of 101766 records.")

unique_patients = diabetes['patient_nbr'].nunique()
print(f"There are {unique_patients} unique patients in the dataset of 101766 records.")

#Now we're ready to do the homework!

#Knowing we are working with unique ENCOUNTERS or PATIENTS might help us decide which things to melt, pivot, etc. 

There are 101766 unique encounters in the dataset of 101766 records.
There are 71518 unique patients in the dataset of 101766 records.


Melt

In [7]:
#I'm going to melt the diagnoses by PATIENT_NBR. (Because unless our objectives for this dataset tell us otherwise, diagnoses SHOULD be more dependent on patient than encounter.)
#In visualization, melting is good for seeing multiple variables/columns on one plot.

dx_pt_melt = pd.melt(diabetes,id_vars='patient_nbr', value_vars=['diag_1', 'diag_2', 'diag_3'],var_name = 'upto_3_Dx', value_name = 'DxCode') 
#value_vars = the target variable names to melt, which contain the values we want, var_name = the NEW variable name comprising the variables we melted, value_name = the meaning of the old cell values in their new relationship to the ID variable
print(dx_pt_melt)

#This would make it very easy to filter by diagnosis ... or by patient! 

        patient_nbr upto_3_Dx  DxCode
0           8222157    diag_1  250.83
1          55629189    diag_1     276
2          86047875    diag_1     648
3          82442376    diag_1       8
4          42519267    diag_1     197
...             ...       ...     ...
305293    100162476    diag_3     458
305294     74694222    diag_3     787
305295     41088789    diag_3     296
305296     31693671    diag_3     998
305297    175429310    diag_3     787

[305298 rows x 3 columns]


Pivot

In [8]:
#Because this dataset has more than 101K rows, I'm struggling to see a situation where I would want to PIVOT this without doing anything else. 
# I think we're going to pivot insulin and A1C result, AND count the number of ENCOUNTERS. And in order to count, I need to use pivot_table instead of pivot.

insulin_a1c_pivot = diabetes[['encounter_id', 'A1Cresult', 'insulin']].copy() #Reduced this to get rid of all the noise.
insulin_a1c_pivot = pd.pivot_table(insulin_a1c_pivot,index='insulin',columns='A1Cresult',aggfunc='count') #index means 'row', columns is self-evident, and aggfunc is performed on the remaining variable. 
print(insulin_a1c_pivot)

          encounter_id            
A1Cresult           >7    >8  Norm
insulin                           
Down               443  1598   603
No                1680  2197  2309
Steady            1210  2737  1558
Up                 479  1684   520


Aggregation

In [9]:
#Let's find the average number (and standard deviation) of lab procedures, medications, and outpatient, emergency, and inpatient encounters across all encounters!

mean_std = diabetes.aggregate({'num_lab_procedures': ['mean', 'std'], #fairly basic
                                'num_medications': ['mean', 'std'],
                                'number_outpatient': ['mean', 'std'],
                                'number_emergency': ['mean', 'std'],
                                'number_inpatient': ['mean', 'std']})
print(mean_std)

      num_lab_procedures  num_medications  number_outpatient  \
mean           43.095641        16.021844           0.369357   
std            19.674362         8.127566           1.267265   

      number_emergency  number_inpatient  
mean          0.197836          0.635566  
std           0.930472          1.262863  


Iteration

In [ ]:
# items(): to iterate over the (key,value) pairs: allows you to iterate over each column as a key-value pair, with the label as the key and the column values as a Series object. This method is consistent with the dictionary-like interface of a DataFrame.
# iterrows(): iterate over the rows as (index,series) pairs: returns an iterator that yields index and row pairs, where each row is represented as a Series object, containing the data in each row.
# itertuples(): iterate over the rows as namedtuples: will return an iterator yielding a named tuple for each row in the DataFrame. The first element of the tuple will be the rows corresponding index value, while the remaining values are the row values. This method is generally faster than iterrows() and preserves the data types of the row elements.

#For this problem only, I am going to ASSUME ALL DATA WERE COLLECTED OVER A YEAR and iterate over demographic data (which SHOULD be pretty immutable) for unique patients as a percentage of the total unique patients (hopefully N = 71518). In my imagination, I have narrowed the timeframe to justify dropping patients whose demographic data is irreconcilable. 
#And I will use ITEMS to iterate. 

dm_demo_var = ['patient_nbr','race','gender','age','weight']
dm_demo = diabetes[dm_demo_var].copy() #Making a smaller dataset to work with. 
#At this time and for this problem only, I'm going to quickly switch the 'Unknown/Invalid' values in 'gender' to '?'. There's only 3 of them. 
dm_demo['gender'] = dm_demo['gender'].replace('Unknown/Invalid', '?')

duplicate_mask = dm_demo[dm_demo.duplicated('patient_nbr',keep = False)] #a mask applied to dm_demo to retain only the duplicated patients
diff_demo_ct = duplicate_mask.groupby('patient_nbr')[['race','gender','age','weight']].nunique().count() 

#We want to see how many of the duplicate pt records have DIFFERENT (nunique) demographics between duplicates, when in theory these SHOULD be immutable characteristics. 
#Exceptions include things like weight or age, or any of these might not be captured during the encounter and might have a ? instead. 

#Well, 16,773 is a lot of duplicate patients with disparate demographic info.
#I'm going to set up an if/elif/else situation, and hopefully the ? value accounts for most of the differences. 

def resolve_demog(dm_demo):
    # Drop duplicate rows within the group, just in case
    unique_rows = dm_demo.drop_duplicates(subset=['race', 'gender', 'age', 'weight'])
    
    if len(unique_rows) == 1: # IF = All columns identical
        return unique_rows.iloc[0]

    cleaned = unique_rows.replace('?', pd.NA) # ELIF = Condition is, every column has at most 1 non-null value. First, Make rows with '?' more pandas-friendly for null values. 
    nunique_non_na = cleaned.nunique(dropna=True) # Next, If each column has only 1 unique non-null value, we can safely collapse AND 'roll up' any ? to a single row with legit values.
    if (nunique_non_na <= 1).all(): 
        collapsed = cleaned.ffill().bfill().iloc[0] # Fill ? with the non-null value
        return collapsed
    
    dm_demo['conflict_flag'] = True # ELSE = Conflicting values remain — send to conflict dataframe
    return dm_demo

results = []
conflicts = []

for pid, dem in dm_demo.groupby('patient_nbr', group_keys=False): #Now I'm going to filter so I have a clean dataset of ONLY unique patient IDs with pretty certain demographics. I am eliminating 'conflicted' patients that have differing values for demographics. 
    resolved = resolve_demog(dem)
    
    if isinstance(resolved, pd.Series):
        results.append(resolved)
    else:
        conflicts.append(resolved)

dm_clean_final = pd.DataFrame(results).reset_index(drop=True) # Combine results
total_nunique = dm_clean_final['patient_nbr'].nunique() #Eliminate any remaning duplicates. 
dm_conflict = pd.concat(conflicts, ignore_index=True) if conflicts else pd.DataFrame() #We don't do anything else with our conflicted patients, but we'll store them nonetheless. 

print(f"Unique or resolved patients: {total_nunique}")
print(f"Conflicting patients: {dm_conflict['patient_nbr'].nunique()}")

#NOW we're FINALLY going to iterate to find percentages for race, gender, age, and weight. 
demo_counts_for_pct = []

for var, col_data in dm_clean_final[['race','gender','age','weight']].items(): #ITEMS, WITH COLUMNS AS KEYS AND COLUMN VALUES AS VALUES (THE PAIR)
    counts = {}
    for val in col_data: #The values in the pair
        counts[val] = counts.get(val, 0) + 1 #the 1 allows us to keep counting as we iterate over val. 
    for category, count in counts.items(): #Now that we've counted everything, we apply .ITEMS again and iterate again, making percentages from our counts. 
        pct = round((count / total_nunique) * 100, 2)
        demo_counts_for_pct.append({'variable': var, 'category': category, 'percent': pct}) #makes a nested list ... 

demo_pct_df = pd.DataFrame(demo_counts_for_pct) #makes the nested list a dataframe! 
print(demo_pct_df)


Unique or resolved patients: 69769
Conflicting patients: 1749
   variable         category  percent
0      race        Caucasian    74.88
1      race  AfricanAmerican    18.06
2      race         Hispanic     2.12
3      race                ?     2.64
4      race            Asian     0.70
5      race            Other     1.62
6    gender           Female    53.13
7    gender             Male    46.86
8    gender                ?     0.00
9       age          [50-60)    17.36
10      age          [80-90)    16.37
11      age          [30-40)     3.75
12      age          [60-70)    22.28
13      age          [40-50)     9.55
14      age          [70-80)    25.43
15      age          [10-20)     0.75
16      age         [90-100)     2.72
17      age          [20-30)     1.57
18      age           [0-10)     0.22
19   weight                ?    95.93
20   weight          [50-75)     1.09
21   weight          [25-50)     0.12
22   weight         [75-100)     1.65
23   weight        [100-12

Groupby

In [21]:
#Now I'm going to count the number of unique patients and .GROUPBY each medication. 

Rx = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
Yes_values = ['Steady', 'Up', 'Down'] # We don't include the 'no' values. ... and we don't care what kind of 'yes' value we get, we're just reducing it to No/Yes. 

mask = diabetes[Rx].isin(Yes_values).any(axis=1) #This mask gets rid of any records that have none of the medications
Rx_filtered = diabetes[mask] #Keeps the records that have any of the medications
Rx_pts_melt = pd.melt(Rx_filtered,id_vars='patient_nbr', value_vars=Rx,var_name = 'Rx', value_name = 'RxYes') 
#We are melting this to get our many medication columns into 1 medication column. 
# And because each patient will have some BUT NOT ALL medications, after melting, we will still have plenty of records with "no" in them. 
# A "no" record means the patient is not on THIS medication but is on something else.
Rx_filtered2 = Rx_pts_melt[Rx_pts_melt['RxYes'].isin(Yes_values)] #So we're going to filter out all the "No"s again.
#This time we have only yesses, but we still have patients represented multiple times ... due to multiple encounters and/or multiple medications.
Rx_pts_agg = (Rx_filtered2.groupby('Rx',as_index = False).agg(num_pts = ('patient_nbr','nunique'))) #FINALLY, we're going to count unique patients (with the .agg function) and group them by medication (with .groupby)
print(Rx_pts_agg)

                          Rx  num_pts
0                   acarbose      236
1              acetohexamide        1
2             chlorpropamide       76
3                glimepiride     4187
4   glimepiride-pioglitazone        1
5                  glipizide     9998
6        glipizide-metformin        9
7                  glyburide     8558
8        glyburide-metformin      567
9                    insulin    39577
10                 metformin    16477
11    metformin-pioglitazone        1
12   metformin-rosiglitazone        2
13                  miglitol       24
14               nateglinide      579
15              pioglitazone     5939
16               repaglinide     1177
17             rosiglitazone     5184
18                tolazamide       33
19               tolbutamide       21
20              troglitazone        3
